## Car Minimum Time

In [ ]:
using TrajectoryOptimization
using Plots, LinearAlgebra

Import the car model

In [ ]:
model = Dynamics.car_model
n = model.n # number of states
m = model.m; # number of controls

dt = 0.1 # discretization time step
model_d = rk4(model,dt); # discrete time model

Define numerical type

In [ ]:
T = Float64;

Define initial and goals states

In [ ]:
x0 = [0.0;0.0;0.0]
xf = [0.0;1.0;0.0];

Define a cost function, e.g., quadratic

In [ ]:
Qf = 100.0*Diagonal(I,n)
Q = (1e-2)*Diagonal(I,n)
R = (1e-2)*Diagonal(I,m)
cost = LQRCost(Q,R,Qf,xf);

Define constraints

In [ ]:
u_bnd = 2.
bnd = bound_constraint(n,m,u_min=-u_bnd,u_max=u_bnd,trim=true)

goal_con = goal_constraint(xf)

con = [bnd, goal_con];

Set options

In [ ]:
verbose=false
opts_ilqr = iLQRSolverOptions{T}(verbose=false,live_plotting=:off)

opts_al = AugmentedLagrangianSolverOptions{T}(verbose=false,opts_uncon=opts_ilqr,
    iterations=30,penalty_scaling=10.0)

opts_altro = TrajectoryOptimization.ALTROSolverOptions{T}(verbose=false,opts_al=opts_al,R_minimum_time=15.0,
    dt_max=0.2,dt_min=1.0e-3);

Define a problem

In [ ]:
N = 51
tf = 3.0
U = [ones(m) for k = 1:N-1]
dt = 0.06
obj = Objective(cost,N)
constraints = TrajectoryOptimization.ProblemConstraints(con,N)

prob = Problem(model_d,Objective(cost,N),U,constraints=TrajectoryOptimization.ProblemConstraints(con,N),dt=dt,x0=x0)
initial_controls!(prob, U);

Solve problem

In [ ]:
solve!(prob, opts_altro) # solve with iLQR

Define minimum time problem

In [ ]:
U = [ones(m) for k = 1:N-1]

prob_mt = Problem(model_d,Objective(cost,N),U,
    constraints=TrajectoryOptimization.ProblemConstraints(con,N),
    dt=dt,x0=x0,tf=:min);

Solve minimum time problem

In [ ]:
solve!(prob_mt, opts_altro);# solve with iLQR

Plot state trajectories

In [ ]:
x = [prob.X[k][1] for k = 1:N]
y = [prob.X[k][2] for k = 1:N]

x_min_time = [prob_mt.X[k][1] for k = 1:N]
y_min_time = [prob_mt.X[k][2] for k = 1:N];

In [ ]:
plot()
# plot_obstacles(circles,:orange)
plot!(x,y,xlabel="x",ylabel="y",label="ALTRO",legend=:right,color=:blue,linestyle=:dash,ratio=:equal,title="state trajectory")
plot!(x_min_time,y_min_time,xlim=(-0.75,0.75),xlabel="x",ylabel="y",label="ALTRO Min. Time.",color=:blue,width=2,ratio=:equal,title="state trajectory")

plot!((x[1],y[1]),marker=:circle,color=:red,label="start")
plot!((x[end],y[end]),marker=:circle,color=:green,label="goal")

Compute total time

In [ ]:
tt = TrajectoryOptimization.total_time(prob)
tt_mt = TrajectoryOptimization.total_time(prob_mt)

Plot control trajectories

In [ ]:
t_range = range(0, stop=tt, length=N)[1:end-1]
u1 = [prob.U[k][1] for k = 1:N-1]
u2 = [prob.U[k][2] for k = 1:N-1];

In [ ]:
t_range_mt = range(0, stop=tt_mt, length=N)[1:end-1]
u1_mt = [prob_mt.U[k][1] for k = 1:N-1]
u2_mt = [prob_mt.U[k][2] for k = 1:N-1];

In [ ]:
plot(title="Control Trajectory",xlabel="time (s)")
plot!(t_range,u1,label="u1")
plot!(t_range,u2,label="u2")
plot!(t_range_mt,u1_mt,width=2,label="u1 (Min. Time)")
plot!(t_range_mt,u2_mt,width=2,label="u2 (Min. Time)")